In [1]:
# ライブラリのインポート
from textblob import TextBlob
import tweepy
import pandas as pd
import nltk
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from googletrans import Translator

In [2]:
# APIの設定
consumerKey = "hCG4WejOO7GhB7b3CzF90kVNb"
consumerSecret = "0JFvkMOS14gaWcL56DOnuTUD7wc1GdqnGxuXXqhOeboPwGdXYm"
accessToken = "1500625576834396161-wjWFXEwvKVtW8lAz20Uy7QjQRRbTEJ"
accessTokenSecret = "BPUYX2rtZLAdiCvqJXnEOyNTTlTgWyEbmudy17iYJf6sC"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
# キーワード、取得ツイート数の入力
keyword = input("ハッシュタグやキーワードを入力してください： ")
noOfTweet = int(input ("取得するツイートの数を入力してください： "))

ハッシュタグやキーワードを入力してください：  日本
取得するツイートの数を入力してください：  100


In [4]:
keyword_list = keyword.split(",")

In [5]:
# リツイートは取得しない
keyword += "-filter:retweets"

In [6]:
# ツイートの取得
tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(noOfTweet)

In [7]:
# 元ツイート
tweet_list_original = pd.DataFrame([tweet.text for tweet in tweets], columns=["original"])
tweet_list_original.head(5)

,original
0,これ持ってセブチちゃんの映画(発声禁止の応援上映?)見ながらぷいきゅあ〜がんばえ〜！！！的な...
1,みんなからの匿名質問を募集中！\n\nこんな質問に答えてるよ\n● 一日中寝てたい・・・凜さ...
2,日本「よくわからん」 https://t.co/H4YAOuCOZe
3,@Jneko0x0 ご参加ありがとうございます！\n当落はCMの最後でご確認ください。\n日...
4,@nyannsuke ご参加ありがとうございます！\n当落はCMの最後でご確認ください。\n...


In [8]:
# 重複削除
tweet_list_original.drop_duplicates()
tweet_list_original.head(5)

,original
0,これ持ってセブチちゃんの映画(発声禁止の応援上映?)見ながらぷいきゅあ〜がんばえ〜！！！的な...
1,みんなからの匿名質問を募集中！\n\nこんな質問に答えてるよ\n● 一日中寝てたい・・・凜さ...
2,日本「よくわからん」 https://t.co/H4YAOuCOZe
3,@Jneko0x0 ご参加ありがとうございます！\n当落はCMの最後でご確認ください。\n日...
4,@nyannsuke ご参加ありがとうございます！\n当落はCMの最後でご確認ください。\n...


In [9]:
# 英語に翻訳
translator = Translator()
tweet_list_translated = [] # 英語に翻訳したツイート
for tweet in tweet_list_original["original"] :
    if detect(tweet)=="en" :
        tweet_list_translated.append(tweet)
    else:
        tweet_list_translated.append(translator.translate(tweet, dest='en').text)

In [10]:
tweet_list_translated = pd.DataFrame(tweet_list_translated)
tweet_list_translated.head(5)

,0
0,"Having this, Cebuchi's movie (Shorting Certifi..."
1,Recruiting anonymous questions from everyone!\...
2,"Japan ""Well I don't understand"" https://t.co/H..."
3,@ Jneko0x0 Thank you for joining!\nPlease chec...
4,@nyannsuke Thank you for your participation!\n...


In [11]:
# ツイート情報
tweet_df = pd.DataFrame()
tweet_df["original"] = tweet_list_original
tweet_df["translated"] = tweet_list_translated
tweet_df.head(5)

,original,translated
0,これ持ってセブチちゃんの映画(発声禁止の応援上映?)見ながらぷいきゅあ〜がんばえ〜！！！的な...,"Having this, Cebuchi's movie (Shorting Certifi..."
1,みんなからの匿名質問を募集中！\n\nこんな質問に答えてるよ\n● 一日中寝てたい・・・凜さ...,Recruiting anonymous questions from everyone!\...
2,日本「よくわからん」 https://t.co/H4YAOuCOZe,"Japan ""Well I don't understand"" https://t.co/H..."
3,@Jneko0x0 ご参加ありがとうございます！\n当落はCMの最後でご確認ください。\n日...,@ Jneko0x0 Thank you for joining!\nPlease chec...
4,@nyannsuke ご参加ありがとうございます！\n当落はCMの最後でご確認ください。\n...,@nyannsuke Thank you for your participation!\n...


In [12]:
# 感情分析
tweet_df[['polarity', 'subjectivity']] = tweet_df['translated'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tweet_df['translated'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']

    if neg > pos: tweet_df.loc[index, 'sentiment'] = "negative"
    elif pos > neg: tweet_df.loc[index, "sentiment"] = "positive"
    else: tweet_df.loc[index, 'sentiment'] = "neutral"
    tweet_df.loc[index, 'neg'] = neg
    tweet_df.loc[index, 'neu'] = neu
    tweet_df.loc[index, 'pos'] = pos
    tweet_df.loc[index, 'compound'] = comp
tweet_df.head(10)

,original,translated,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,これ持ってセブチちゃんの映画(発声禁止の応援上映?)見ながらぷいきゅあ〜がんばえ〜！！！的な...,"Having this, Cebuchi's movie (Shorting Certifi...",-0.390625,0.400000,negative,0.180,0.820,0.000,-0.7984
1,みんなからの匿名質問を募集中！\n\nこんな質問に答えてるよ\n● 一日中寝てたい・・・凜さ...,Recruiting anonymous questions from everyone!\...,0.450000,0.400000,positive,0.000,0.845,0.155,0.7840
2,日本「よくわからん」 https://t.co/H4YAOuCOZe,"Japan ""Well I don't understand"" https://t.co/H...",0.000000,0.000000,positive,0.000,0.656,0.344,0.2732
3,@Jneko0x0 ご参加ありがとうございます！\n当落はCMの最後でご確認ください。\n日...,@ Jneko0x0 Thank you for joining!\nPlease chec...,0.157273,0.410909,positive,0.000,0.693,0.307,0.8745
4,@nyannsuke ご参加ありがとうございます！\n当落はCMの最後でご確認ください。\n...,@nyannsuke Thank you for your participation!\n...,0.157273,0.410909,positive,0.000,0.700,0.300,0.8745
5,マスコミが「日韓関係が冷え込んでいる」って言うの、いつもすごい引っかかる。日本は淡々と事実は...,It is always very surprised to say the mass me...,-0.140833,0.736111,negative,0.094,0.826,0.079,-0.2280
6,@KoU20130712 ご参加ありがとうございます！\n当落はCMの最後でご確認ください。...,@ Kou20130712 Thank you for joining!\nPlease c...,0.157273,0.410909,positive,0.000,0.693,0.307,0.8745
7,アメブロを投稿しました。\n『今夜の餌 BGM by 村治佳織』\n#アメブロ https:...,"I posted an Amebro.\n""Tonight bait BGM by Mura...",-0.041667,0.420833,neutral,0.000,1.000,0.000,0.0000
8,@ten_ten_tenko10 お疲れさまでした\n毎日ウクライナ戦争報道で、次は欧州、日...,@ ten_ten_tenko10 Thank you very much\nEvery d...,0.130000,0.130000,negative,0.303,0.515,0.182,-0.6124
9,@gbvCsmKSGLOoT7f ご参加ありがとうございます！\n当落はCMの最後でご確認く...,@ GBVCSMKSGLOOT7F Thank you for your participa...,0.157273,0.410909,positive,0.000,0.700,0.300,0.8745


In [13]:
# sentimentのグループごとに数と割合を返す関数
def count_values_in_column(data,feature):
 total=data.loc[:,feature].value_counts(dropna=False)
 percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
 return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [14]:
count_values_in_column(tweet_df, "sentiment")

,Total,Percentage
positive,57,57.0
negative,23,23.0
neutral,20,20.0


In [15]:
# 否定的（アンチ）なツイートだとみなされたツイートを詳細に見る例
for index, tweet in tweet_df[tweet_df["sentiment"]=="negative"].iterrows():
    print(tweet.original)
    print(tweet)
    print("-----------------------------------------------------------------------------------")

これ持ってセブチちゃんの映画(発声禁止の応援上映?)見ながらぷいきゅあ〜がんばえ〜！！！的なことをすれば良いんですか？映画日本上陸までにそれまでに届けてくれますか？？？？？？？？ https://t.co/LXFuv5ngog
original        これ持ってセブチちゃんの映画(発声禁止の応援上映?)見ながらぷいきゅあ〜がんばえ〜！！！的な...
translated      Having this, Cebuchi's movie (Shorting Certifi...
polarity                                                -0.390625
subjectivity                                                  0.4
sentiment                                                negative
neg                                                          0.18
neu                                                          0.82
pos                                                           0.0
compound                                                  -0.7984
Name: 0, dtype: object
-----------------------------------------------------------------------------------
マスコミが「日韓関係が冷え込んでいる」って言うの、いつもすごい引っかかる。日本は淡々と事実はこうですって言ってるだけなのに、韓国が嘘と捏造で被害者ビジネスに仕立てて大騒ぎしてるだけだろう。日本はなんもしてないのに日本が悪いスタンスを含んでる言葉ですごく不愉快
original        マスコミが「日韓関係が冷え込んでいる」って言うの、いつもすごい引っかかる。日本

In [16]:
# アンチコメント判定の目安
# for index, row in tweet_df[tweet_df["sentiment"]=="negative"][tweet_df["neg"]>0.25].iterrows():
#     print(row)